In [3]:
import pandas as pd
import plotly.express as px

# Загрузка данных
path = '''C:\\Personal\\Uni\\5 сем\\Информационные системы и технологии\\ИСиТ_КР.xlsm'''
df = pd.read_excel(path, sheet_name='Журнал учета', skiprows=5)

df['Курс'] = df['Номер группы'].astype('str') + ", " + df["Предмет"]


# Создание диаграммы
fig = px.timeline(df,
                  x_start="Дата начала курса",
                  x_end="Дата окончания курса",
                  y='Курс',
                  color="ФИО",
                  title="Расписание курсов",
                  labels={"Тип занятия": "Тип занятия", "ФИО": "Преподаватель"},
                  color_discrete_map={"лекция": "blue", "практика": "green"},
                  template='plotly_white'
                  )

# Настройка внешнего вида
fig.update_yaxes(categoryorder="category descending",)
fig.update_layout(
    width=1200,
    height=800,
    font=dict(size=12),
    title_font_size=20,
    xaxis_title="Период проведения курса",
    yaxis_title="Курс",
    legend_title="Преподаватель")

# Отображение диаграммы
fig.show()
